In [5]:
# %load std_libraries.py
#to keep my scripts consistent, I am adding this to the top of all scripts with %load std_libraries.py
import sys
#Indicate operating environment and import core modules
location_input = input("what computer are you on? a = Bens, b = gpucluster, c = other   ")
location_dict = {'a': "C:\\Users\\heine\\github\\expression_broad_data", 'b': "/home/heineike/github/expression_broad_data",'c':'you need to add your location to the location_dict'}
base_dir = location_dict[location_input]
print("base directory is " + base_dir)

if sys.path[-1] != base_dir:
    sys.path.append(base_dir)
    print("Added " + base_dir + " to path: " )
    print(sys.path)

import os

print("I am about to import a library")
from core import expression_plots 
from core import io_library 
%load_ext autoreload
%autoreload 2
# %matplotlib 
# plt.ioff()
#%matplotlib inline
%matplotlib notebook
data_processing_dir = base_dir + os.sep + os.path.normpath("expression_data") + os.sep

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns; sns.set(style="ticks", color_codes=True)
#from sklearn import linear_model
#import pickle
#import subprocess
#import networkx as nx
# import scipy.stats as stats
import scipy.cluster.hierarchy as sch

from Bio import SeqIO
from Bio import SeqFeature as sf
from Bio.Alphabet import generic_dna
from Bio.Seq import Seq

import re

from collections import Counter
# import scipy.stats as stats
# from itertools import chain
#this only works if you are online
online_input = input("are you online? Yes/No")
if online_input == "Yes": 
    import plotly.plotly as py
    import plotly.graph_objs as pygo
    import plotly.tools as pytools
    py.sign_in('heineike02_student','9dMTMZgJMgUP0YX0P5mQ')
    #py.sign_in('heineike02', 'APPjKrtARaN2ZgUYIkqr')
    
# for phylogenetic trees: 
from ete3 import Tree

#for scraping internet data (e.g. ncbi)
import requests
from lxml import etree    #parses xml output


what computer are you on? a = Bens, b = gpucluster, c = other   a
base directory is C:\Users\heine\github\expression_broad_data
I am about to import a library
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
are you online? Yes/NoYes


In [ ]:
#Get promoters for all the genes I am interested in

#Calculate 
#   Presence + location of exact STRE
#   Presence + location of core PDS
#   Presence (using motif?) of TATA box

#Display
#   Numbers of elements for all gois
#   Image with location on promoter for any given goi



In [2]:
#load species info table
species_info = pd.read_csv(os.path.normpath(data_processing_dir + 'promoter_phylogenies/ygob_species_info.csv'))

In [2]:
#load phylogenentic tree 
sc_genes = ['GND1', 'TAL1', 'SOL3']
sc_gene = 'TKL1'
fname = os.path.normpath(data_processing_dir + 'promoter_phylogenies/' + sc_gene + '_tree_ygob.txt')
with open(fname,'r') as f: 
    gene_phylogeny = f.read()

tree = Tree(gene_phylogeny)
print(tree)


      /-Kpol_1039.57
   /-|
  |   \-TPHA0M01660
  |
  |      /-KAFR0D02620
  |     |
  |   /-|      /-NCAS0I01030
  |  |  |   /-|
  |  |  |  |   \-NDAI0A07980
  |  |   \-|
  |  |     |   /-KNAG0B01420
  |  |      \-|
  |  |        |   /-Suva_4.364
--|  |         \-|
  |  |           |   /-Smik_2.257
  |  |            \-|
  |  |              |   /-YBR117C
  |  |               \-|
  |  |                  \-Skud_2.244
  |  |
  |  |            /-Kwal_23.3955
  |  |         /-|
  |  |        |   \-KLTH0F11836g
  |  |      /-|
  |  |     |  |   /-SAKL0B08096g
  |  |     |   \-|
   \-|     |     |   /-KLLA0B09152g
     |     |      \-|
     |     |        |   /-ADL366W
     |     |         \-|
     |     |            \-Ecym_3454
     |     |
     |     |                  /-NDAI0A05060
     |     |               /-|
     |     |            /-|   \-NCAS0A11230
     |     |           |  |
     |     |           |   \-KAFR0H00840
     |   /-|         /-|
     |  |  |        |  |      /-Skud_16.3

In [3]:
species_info = pd.read_csv(os.path.normpath(data_processing_dir + 'promoter_phylogenies/ygob_species_info.csv'))
species_info
#species_info[~species_info['ygob_gene_prefix'].isna()].loc[:,['ygob_gene_prefix','abbreviation']]

,abbreviation,ygob_gene_prefix,ncbi_period_convert,ncbi_name,in_ncbi,in_sss,in_genolevures
0,Vpol,Kpol,True,Vanderwaltozyma polyspora,True,False,False
1,Tpha,TPHA0,False,Tetrapisispora phaffii,True,False,False
2,Tbla,TBLA0,False,Tetrapisispora blattae,True,False,True
3,Ndai,NDAI0,False,Naumovozyma dairenensis,True,False,False
4,Ncas,NCAS0,False,Naumovozyma castellii,True,False,True
5,Knag,KNAG0,False,Kazachstania naganishii,True,False,True
6,Kafr,KAFR0,False,Kazachstania africana,True,False,False
7,Cagl,CAGL0,False,Candida glabrata,True,False,False
8,Suva,Suva,False,Saccharomyces uvarum,False,True,False
9,Skud,Skud,False,Saccharomyces kudriavzevii,False,True,False


In [46]:
a = gene_data_xml.getchildren()[0]
b = a.find('Entrezgene_locus')
b != None

True

In [47]:
a.tag

'Entrezgene'

In [24]:
entrez_gene_id = '5544171'
gene_data_url = 'http://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=gene&id=' + entrez_gene_id + '&retmode=xml'
gene_data_response = requests.get(gene_data_url) 
gene_data_xml = etree.fromstring(gene_data_response.content)
entrezgene_element = gene_data_xml.getchildren()[0]
locus_element = entrezgene_element.find('Entrezgene_locus')
#check that Entrezgene_locus is not none
if locus_element == None: 
    raise ValueError('Entrezgene_locus not found ' + gene)
locus_gene_commentary_list = locus_element.findall("Gene-commentary") 
if len(locus_gene_commentary_list) == 1: 
    locus_gene_commentary = locus_gene_commentary_list[0]
else: 
    print('NCBI has more than one gene commentary elements in the Entrezgene Locus looking for type 1 = genomic')
    #Keep only the gene commentary that has type = 1 
    locus_gene_commentary_shortlist = [locus_gene_commentary for locus_gene_commentary in locus_gene_commentary_list if locus_gene_commentary.find("Gene-commentary_type").text=='1']
    if len(locus_gene_commentary_shortlist) == 1: 
        locus_gene_commentary = locus_gene_commentary_shortlist[0]
    else: 
        print('Error - more than one gene commentary listed for genomic dna.  gene = ' + gene)
if locus_gene_commentary.find("Gene-commentary_type").text != '1':
    print('Error - gene commentary type is not 1: genomic dna.  gene = ' + gene)

ValueError: Entrezgene_locus not at position 7

In [ ]:
#Intergenic regions for the four sensu stricu species is here: 
#http://www.saccharomycessensustricto.org/cgi-bin/s3.cgi?data=Intergenics&version=current

#genolevures has the other missing species. 
#NOt to hard to get the promoter from the web interface here - 
#wish there was an API though
#gene_search_url = 'http://gryc.inra.fr/index.php?page=locus&seqid=SAKL0B08096g'
#have to add it to cart, check nucleotide and then download after adding 1000bases to the 5' site.

In [49]:
#load species info table
species_info = pd.read_csv(os.path.normpath(data_processing_dir + 'promoter_phylogenies/ygob_species_info.csv'))
sc_genes = ['GND1', 'TAL1', 'SOL3']

for sc_gene in sc_genes: 
    fname = os.path.normpath(data_processing_dir + 'promoter_phylogenies/' + sc_gene + '_tree_ygob.txt')
    with open(fname,'r') as f: 
        gene_phylogeny = f.read()
    tree = Tree(gene_phylogeny)   

    genes = tree.get_leaf_names()
    prom_length = 1000
    specs = []
    promoter_outputs = {}
    #go get its promoter sequence. 
    for gene in genes: 
        spec, promoter_output = io_library.extract_promoter_sequence(gene,prom_length)
        specs.append(spec)
        if isinstance(promoter_output,tuple):
            promoter_outputs[gene] = promoter_output

    promoter_data = pd.DataFrame.from_dict(promoter_outputs, orient='index')
    promoter_data.columns =  ["refseq","loc","strand","prom_seq","gene_seq"]
    promoter_data.index.name = 'genename'

    gene_list = list(promoter_data.index)
    fname = os.path.normpath(data_processing_dir + 'promoter_phylogenies/ortholog_promoters_' + sc_gene + '_' + str(prom_length) + '.fasta')
    #Changed name from write_ame_promoter_file to write_promoter_file 10JAN18    
    io_library.write_promoter_file(promoter_data, gene_list,fname)



YHR183W
Suva_15.384 not in NCBI - no method to get promoter yet
Skud_8.246 not in NCBI - no method to get promoter yet
Smik_8.271 not in NCBI - no method to get promoter yet
YGR256W
Skud_7.590 not in NCBI - no method to get promoter yet
Suva_7.554 not in NCBI - no method to get promoter yet
AFR129W
NCBI has more than one gene commentary elements in the Entrezgene Locus looking for type 1 = genomic
SAKL0H01848g not in NCBI - no method to get promoter yet
Kwal_23.4361 not in NCBI - no method to get promoter yet
ACL196W
NCBI has more than one gene commentary elements in the Entrezgene Locus looking for type 1 = genomic
Kwal_14.1439 not in NCBI - no method to get promoter yet
SAKL0B04642g not in NCBI - no method to get promoter yet
YLR354C
Skud_12.439 not in NCBI - no method to get promoter yet
Suva_10.455 not in NCBI - no method to get promoter yet
Smik_12.442 not in NCBI - no method to get promoter yet
Suva_7.328 not in NCBI - no method to get promoter yet
Skud_7.340 not in NCBI - no met

In [4]:
#For a given promoter data file, do a fimo search 

sc_genes = ['GND1', 'TAL1', 'SOL3', 'TKL1']

sc_gene_phylogenies = {}

ncbi_specs = species_info[species_info['in_ncbi']]['abbreviation']


for sc_gene in sc_genes:

    #Load the fimo search data 
    fimo_fname = os.path.normpath(data_processing_dir + 'promoter_phylogenies/fimo_' + sc_gene + '_orthologs.txt')
    fimo_hits = pd.read_table(fimo_fname)
    
    #Load the genes: 
    fname = os.path.normpath(data_processing_dir + 'promoter_phylogenies/' + sc_gene + '_tree_ygob.txt')
    with open(fname,'r') as f: 
        gene_phylogeny = f.read()
    tree = Tree(gene_phylogeny)  

    #filter out the genes that didn't have promoters to search
    genes = tree.get_leaf_names()
    genes_ncbi = [gene for gene in genes if io_library.get_species_from_gene(gene, species_info) in set(ncbi_specs)]

    motifs = list(set(fimo_hits['# motif_id']))

    ortholog_motifs = pd.DataFrame(index = genes_ncbi)


    length_filter = 700

    print('this lenght filter is wrong: low numbers are farther away from the promoter - need to invert it') 
    for motif in motifs: 
        fimo_hits_motif = fimo_hits[fimo_hits['# motif_id']==motif]
        #Filter motif to start site less than length_filter
        fimo_hits_motif_lfilter = fimo_hits_motif[fimo_hits_motif['start']<length_filter]
        ortholog_motifs[motif] = [Counter(fimo_hits_motif_lfilter['sequence_name'])[gene] for gene in genes_ncbi]

    sc_gene_phylogenies[sc_gene] = ortholog_motifs
    

YHR183W
YGR256W
AFR129W
ACL196W
YLR354C
YGR043C
AEL215C
YHR163W
YGR248W
YBR117C
ADL366W
YPR074C


In [6]:
#Load the genes: 
sc_gene = 'TKL1'
fname = os.path.normpath(data_processing_dir + 'promoter_phylogenies/' + sc_gene + '_tree_ygob.txt')
with open(fname,'r') as f: 
    gene_phylogeny = f.read()
tree = Tree(gene_phylogeny) 
print(tree)


      /-Kpol_1039.57
   /-|
  |   \-TPHA0M01660
  |
  |      /-KAFR0D02620
  |     |
  |   /-|      /-NCAS0I01030
  |  |  |   /-|
  |  |  |  |   \-NDAI0A07980
  |  |   \-|
  |  |     |   /-KNAG0B01420
  |  |      \-|
  |  |        |   /-Suva_4.364
--|  |         \-|
  |  |           |   /-Smik_2.257
  |  |            \-|
  |  |              |   /-YBR117C
  |  |               \-|
  |  |                  \-Skud_2.244
  |  |
  |  |            /-Kwal_23.3955
  |  |         /-|
  |  |        |   \-KLTH0F11836g
  |  |      /-|
  |  |     |  |   /-SAKL0B08096g
  |  |     |   \-|
   \-|     |     |   /-KLLA0B09152g
     |     |      \-|
     |     |        |   /-ADL366W
     |     |         \-|
     |     |            \-Ecym_3454
     |     |
     |     |                  /-NDAI0A05060
     |     |               /-|
     |     |            /-|   \-NCAS0A11230
     |     |           |  |
     |     |           |   \-KAFR0H00840
     |   /-|         /-|
     |  |  |        |  |      /-Skud_16.3

In [75]:
sc_genes = ['TAL1', 'SOL3', 'TKL1']

phylogeny_pairs = {'YPR074C': 'KNAG0A07310', 
                   'YBR117C':'KAFR0D02620'}

track1_track2 = {'TKL1':'TKL2',
                 #'GND1':'GND2',
                 'TAL1':'NQM1',
                 'SOL3':'SOL4'}

sc_gene = 'TKL1'

fname = os.path.normpath(data_processing_dir + 'promoter_phylogenies/' + sc_gene + '_tree_ygob.txt')
with open(fname,'r') as f: 
    gene_phylogeny = f.read()
tree = Tree(gene_phylogeny) 

track1_ancestor = tree.get_common_ancestor(['YBR117C',phylogeny_pairs['YBR117C']])
track1_orthologs = track1_ancestor.get_leaf_names()
#remove seed_outgroup
#track1_orthologs.remove(seed_outgroup)
track1_orthologs_ncbi = [gene for gene in track1_orthologs if io_library.get_species_from_gene(gene, species_info) in set(ncbi_specs)]
track1_orthologs_ncbi_spec = {io_library.get_species_from_gene(gene, species_info): gene for gene in track1_orthologs_ncbi}
#organize in species order
#post_wgd = ['Scer','Cagl','Kafr','Knag','Ncas','Ndai','Tbla','Tpha','Vpol']

[track1_orthologs_ncbi_spec[spec] for spec in post_wgd]

YBR117C
YBR117C


KeyError: 'Cagl'

In [34]:
track1 = {}
track2 = {}

#T1 = TKL1, GND1, TAL1, SOL3
#T2 = TKL2, GND2, NQM1, SOL4


track1['GND1'] = ['YHR183W','CAGL0M13343g','KAFR0B01610','TBLA0C06670','NCAS0F02030', 'NDAI0C03490', 'Kpol_1067.2']
track2['GND1'] = ['YGR256W','KAFR0B04490','KNAG0K00390','TPHA0A05350']

track1['SOL3'] = ['YHR163W','TPHA0A05240', 'CAGL0I02200g', 'KNAG0E00640', 'Kpol_1050.78', 'NCAS0D01430', 'NDAI0C03310', 'KAFR0E02210' ]
track2['SOL3'] = ['YGR248W', 'KAFR0B04400', 'KNAG0K00490', 'NCAS0E03860', 'NDAI0B00950', 'TBLA0C06580' ]

track1['TAL1'] = ['YLR354C', 'NDAI0J01880', 'NCAS0J01140', 'KNAG0B05780', 'KAFR0A06290','CAGL0B03069g','Kpol_472.17', 'TPHA0F01080', 'TBLA0A08400']
track2['TAL1'] = ['YGR043C', 'CAGL0K04235g', 'KNAG0D04010', 'KAFR0F03930', 'NDAI0D04080', 'NCAS0A02150', 'Kpol_530.51', 'TPHA0J00660']

track1['TKL1'] = ['YPR074C','CAGL0D01298g', 'KAFR0H00840', 'KNAG0A07310', 'NCAS0A11230', 'NDAI0A05060', 'TBLA0H01480', 'TPHA0A03490','Kpol_1039.57']
track2['TKL1'] = ['YBR117C','KAFR0D02620', 'KNAG0B01420','NCAS0I01030','NDAI0A07980', 'TPHA0M01660', 'Kpol_1006.8']

In [35]:

#Make dataframe with index 
ncbi_specs = species_info[species_info['in_ncbi']]['abbreviation']

pre_wgd = ['Klac','Lthe', 'Egos','Ecym','Tdel','Zrou']
pre_wgd_ncbi = [spec for spec in pre_wgd if spec in set(ncbi_specs)]
post_wgd = ['Scer','Cagl','Kafr','Knag','Ncas','Ndai','Tbla','Tpha','Vpol']
post_wgd_ncbi = [spec for spec in post_wgd if spec in set(ncbi_specs)]

post_wgd_t1 = [spec + '_t1' for spec in post_wgd_ncbi]
post_wgd_t2 = [spec + '_t2' for spec in post_wgd_ncbi]

spec_index = post_wgd_t1 + post_wgd_t2 + pre_wgd_ncbi

sc_genes = ['TAL1', 'SOL3', 'TKL1', 'GND1']

orthologs_table = pd.DataFrame(index = spec_index, columns = sc_genes)


for sc_gene in sc_genes: 
    #make a list with a gene or None for each species that we searched in FIMO in the order of the index. 
    gene_list = []
    for track in [track1, track2]: 
#         print(sc_gene)
#         print(track)
#         print(track[sc_gene])
        track_sc_gene = track[sc_gene]
        #keep only the genes in NCBI
        track_ncbi = [gene for gene in track_sc_gene if io_library.get_species_from_gene(gene, species_info) in set(ncbi_specs)]
        track_ncbi_spec = {io_library.get_species_from_gene(gene, species_info): gene for gene in track_ncbi}
    
        #for each species in the post_wgd list, try to slot in a gene from the track.  If none is present put in None. 
        for spec in post_wgd_ncbi:
            if spec in track_ncbi_spec.keys(): 
                gene_list.append(track_ncbi_spec[spec])
            else:
                gene_list.append(None)
    
    #add on the species with only one track
    
    fname = os.path.normpath(data_processing_dir + 'promoter_phylogenies/' + sc_gene + '_tree_ygob.txt')
    with open(fname,'r') as f: 
        gene_phylogeny = f.read()
    tree = Tree(gene_phylogeny) 
    orthologs = tree.get_leaf_names()
    
    pre_wgd_orthologs = {}
    for ortholog in orthologs: 
        spec = io_library.get_species_from_gene(ortholog,species_info)
        if spec in set(pre_wgd_ncbi): 
            pre_wgd_orthologs[spec] = ortholog        
            
    for spec in pre_wgd_ncbi: 
        if spec in pre_wgd_orthologs.keys(): 
            gene_list.append(pre_wgd_orthologs[spec])
        else:
            gene_list.append(None)
    
    orthologs_table[sc_gene] = gene_list


orthologs_table
        







YLR354C
YLR354C
YGR043C
YGR043C
ACL196W
YLR354C
YGR043C
YHR163W
YHR163W
YGR248W
YGR248W
AEL215C
YHR163W
YGR248W
YPR074C
YPR074C
YBR117C
YBR117C
YBR117C
ADL366W
YPR074C
YHR183W
YHR183W
YGR256W
YGR256W
YHR183W
YGR256W
AFR129W


,TAL1,SOL3,TKL1,GND1
Scer_t1,YLR354C,YHR163W,YPR074C,YHR183W
Cagl_t1,CAGL0B03069g,CAGL0I02200g,CAGL0D01298g,CAGL0M13343g
Kafr_t1,KAFR0A06290,KAFR0E02210,KAFR0H00840,KAFR0B01610
Knag_t1,KNAG0B05780,KNAG0E00640,KNAG0A07310,None
Ncas_t1,NCAS0J01140,NCAS0D01430,NCAS0A11230,NCAS0F02030
Ndai_t1,NDAI0J01880,NDAI0C03310,NDAI0A05060,NDAI0C03490
Tbla_t1,TBLA0A08400,None,TBLA0H01480,TBLA0C06670
Tpha_t1,TPHA0F01080,TPHA0A05240,TPHA0A03490,None
Vpol_t1,Kpol_472.17,Kpol_1050.78,Kpol_1039.57,Kpol_1067.2
Scer_t2,YGR043C,YGR248W,YBR117C,YGR256W


In [33]:
binding_site_lookup = dict(sc_gene_phylogenies['SOL3']['MSN2_Badis'])
binding_site_lookup

{'AEL215C': 0,
 'CAGL0I02200g': 0,
 'Ecym_5572': 3,
 'KAFR0B04400': 1,
 'KAFR0E02210': 0,
 'KLLA0A05390g': 0,
 'KLTH0H00836g': 2,
 'KNAG0E00640': 2,
 'KNAG0K00490': 1,
 'Kpol_1050.78': 0,
 'NCAS0D01430': 0,
 'NCAS0E03860': 1,
 'NDAI0B00950': 0,
 'NDAI0C03310': 4,
 'TBLA0C06580': 1,
 'TDEL0G01020': 0,
 'TPHA0A05240': 0,
 'YGR248W': 0,
 'YHR163W': 0,
 'ZYRO0C11484g': 1}

In [31]:
binding_site_lookup = dict(sc_gene_phylogenies['SOL3']['MSN2_Badis'])
binding_site_list = []
for ortholog in orthologs_table['SOL3']: 
    if ortholog==None: 
        binding_site_list.append(None)
    else: 
        binding_site_list.append(binding_site_lookup[ortholog])
binding_site_list

[0, 7, 0, 2, 0, 3, 1, 0, 0, 1, 0, 0, 3, 1, 1, None, 1, 1, 1, 1, 2, 4, 0, 0]

In [37]:
binding_site_table = pd.DataFrame(index = spec_index, columns = sc_genes)

for sc_gene in sc_genes: 
    # make a dictionary that links name to number of Msn2
    # use that to make a new column for a new dataframe with the same index 
    binding_site_lookup = dict(sc_gene_phylogenies[sc_gene]['MSN2_Badis'])
    binding_site_list = []
    for ortholog in orthologs_table[sc_gene]: 
        if ortholog==None: 
            binding_site_list.append(None)
        else: 
            binding_site_list.append(binding_site_lookup[ortholog])
    binding_site_list
    binding_site_table[sc_gene] = binding_site_list

binding_site_table.to_csv(os.path.normpath(data_processing_dir + 'promoter_phylogenies/PPP_phylogenies_badis_700.csv'))


## Use YGOB and Trevors intergenics to make orthologous promoters .

In [ ]:
#For a given gene: 

#Given a species list from the YGOB Pillars

    #get set of orthologous genes

#make fasta of orthologous promoters. 
#export file. 


#count no of STREs, PDS's.  Output surrounding 2 bases and locations
#Align and get all conserved regions

#visualize list.

In [6]:
#Make a dataframe based on the YGOB pillars file

fname_pillars = os.path.normpath(data_processing_dir + 'ortholog_files_YGOB\YGOB_pillars_bmh.txt')
YGOB_pillars = pd.read_table(fname_pillars, header = None)

post_wgd = ['Vpol', 'Tpha', 'Tbla', 'Ndai', 'Ncas', 'Knag', 'Kafr', 'Cgla', 'Suva', 'Skud', 'Smik','Scer']
post_wgd_p1 = [spec + '_p1' for spec in post_wgd]
post_wgd_p2 = [spec + '_p2' for spec in post_wgd]
post_wgd_p2.reverse()
pre_wgd = ['Zrou', 'Tdel','Klac','Egos','Ecym','Lklu','Lthe','Lwal']
columns = post_wgd_p1 + ['Anc'] + pre_wgd + post_wgd_p2

YGOB_pillars.columns = columns


#    0    V. polyspora Position 1
#    1    T. phaffii Position 1
#    2    T. blattae Position 1
#    3    N. dairenensis Position 1
#    4    N. castellii Position 1
#    5    K. naganishii Position 1
#    6    K. africana Position 1
#    7    C. glabrata Position 1
#    8    S. bayanus var. uvarum Position 1
#    9    S. kudriavzevii Position 1
#    10   S. mikatae Position 1
#    11   S. cerevisiae Position 1
#    12   Ancestral Gene Order
#    13   Z. rouxii
#    14   T. delbrueckii
#    15   K. lactis
#    16   E. gossypii 
#    17   E. cymbalariae
#    18   L. kluyveri
#    19   L. thermotolerans
#    20   L. waltii
#    21   S. cerevisiae Position 2
#    22   S. mikatae Position 2
#    23   S. kudriavzevii Position 2
#    24   S. bayanus var. uvarum Position 2
#    25   C. glabrata Position 2
#    26   K. africana Position 2
#    27   K. naganishii Position 2
#    28   N. castellii Position 2
#    29   N. dairenensis Position 2
#    30   T. blattae Position 2
#    31   T. phaffii Position 2
#    32   V. polyspora Position 2

In [9]:
gois_gr1_m24 = pd.read_csv(data_processing_dir + '20170817_klscpka/20180327_msn24_gis1_rph1_overlap.csv')
genes = io_library.SC_orf_lookup_by_name(gois_gr1_m24['SC_common_name'])

S.Cer orf for YJR115Wnot found
S.Cer orf for YHR097Cnot found
S.Cer orf for YBR241Cnot found
S.Cer orf for YNL194Cnot found
S.Cer orf for YMR196Wnot found
S.Cer orf for YJL045Wnot found
S.Cer orf for YBR139Wnot found


In [10]:
#for a given list of genes:
#Searches for exact matches of defined motifs in various sequences
motif_dict = {'STRE':'AGGGG',
             'Msn2_pref': 'CGGGG', 
              #'PDS1': 'TTAGGGAT',
              #'PDS2': 'TAAGGGAT',
              'PDS core': 'AGGGA'
             }
output_format = 'full'
sequence_context = 2

output_dict = {}
gene_names = io_library.SC_common_name_lookup(genes) #list(gois_gr1_m24['SC_common_name'])
for ind, gene in enumerate(genes): 
    gene_name = gene_names[ind]
    print(gene_name)
    #Lookup orthologs based on the YGOB Pillars dataframe
    gene_pos = ''
    for pos in ['p1','p2']: 
        Scer_pos = YGOB_pillars['Scer_' + pos]
        Scer_pos_find = Scer_pos[Scer_pos==gene]
        if len(Scer_pos_find)==1:
            if gene_pos != '':
                print('Warning: found gene in two pillars')
            gene_pos = pos
            gene_index = Scer_pos_find.index[0]
        elif len(Scer_pos_find)>1: 
            print('Warning: found gene twice in same pillar')

    #extract intergenic regions from trevor's file

    specs = ['Kafr','Cgla','Suva', 'Skud', 'Smik','Scer']
    ##to include pre wgd species would need to update
    specs_pillar = [spec + '_' + gene_pos for spec in specs]

    orthologs = YGOB_pillars.loc[gene_index, specs_pillar]


    seqs = {}

    for spec in specs: 
        fname_intergenics = os.path.normpath(data_processing_dir + 'promoter_phylogenies/intergenics/' + spec + '_intergenics.faa' )
        record_iterator = SeqIO.parse(fname_intergenics, "fasta")

        for seq in record_iterator: 
            if seq.id == orthologs[spec + '_' + gene_pos]:
                seqs[spec] = seq

    #initial dataframe has species for index, but reindex on seq id before outputting
    seq_ids = [seq.id for seq in seqs.values()]
    output_data_frame = pd.DataFrame(index = seq_ids)
    output_data_frame['spec'] = seqs.keys()

    for motif_name, motif in motif_dict.items():
        L_motif = len(motif)
        output_motif = []

        for spec, seq in seqs.items(): 

            prom_seq = seq.seq
            if len(prom_seq)<no_prom_thresh: 
                output_motif.append('no_promoter')
            else: 
                output_motif_seq = io_library.exact_promoter_scan(motif, prom_seq, output_format, sequence_context)
                output_motif.append(output_motif_seq)
        output_data_frame[motif_name] = output_motif

    output_dict[gene_name] = output_data_frame


HXT5
NDE2
PIR3
GND2
RGI1
USV1
SOL4
TSA2
DBP1
YJR115W
YHR097C
GND1
HSP150
NDE1
HXT3
RGI2
DED1
PAL1
ECM13
SOL3
RGM1
TSA1
GSY2
YBR241C
GLG1
MRK1
ATG33
FMP45
TSL1
PGM2
GSY1
YNL194C
VPS73
RIM11
TPS3
GLG2
SCM4
PGM1
HSP12
GOR1
YMR196W
MBR1
ISF1
PEX18
SHH4
SSE2
YJL045W
SHH3
PEX21
SDH3
SDH1
SDH4
SSE1
YBR139W
SDS24
SDS23
CYC7
HXK1
UBI4
CTT1
GSP2
HXK2
GSP1
CYC1
GIS1
DOT6
RPH1
TOD6


In [45]:
#Load my data and Msn2/4 delete data


In [11]:

full_output = pd.DataFrame()
category_dict = dict(zip(gois_gr1_m24['SC_common_name'],gois_gr1_m24['category']))
kl_sc_PKA_data_solis = pd.read_csv(data_processing_dir + os.path.normpath('20170817_klscpka/kl_sc_PKA_data_solis.csv'))
kl_genename_dict = dict(zip(kl_sc_PKA_data_solis['SC_common_name'],kl_sc_PKA_data_solis['kl_genename']))

for gene_name in gois_gr1_m24['SC_common_name']: 
       
    new_output = output_dict[gene_name]
    new_output['SC_common_name'] = gene_name
    new_output['category'] = category_dict[gene_name]
    new_output['kl_genename'] = kl_genename_dict[gene_name]

    full_output = pd.concat([full_output, new_output])

full_output.to_csv(data_processing_dir + 'promoter_phylogenies/gis1_rph1_STRE_PDS_hits.csv')

In [4]:
full_output = pd.read_csv(data_processing_dir + 'promoter_phylogenies/gis1_rph1_STRE_PDS_hits.csv')

In [5]:
full_output

,Unnamed: 0,spec,STRE,Msn2_pref,PDS,SC_common_name,category,kl_genename
0,CAGL0A02321g,Cgla,"[(884, 'rev', 'GGAGGGGGG')]","[(691, 'rev', 'CTCGGGGTG')]","[(230, 'rev', 'GGAGGGAGG'), (789, 'rev', 'AGAG...",HXT5,g1r1_m24_sc_para,KLLA0D13310g
1,Suva_15.291,Suva,"[(635, 'fwd', 'CGAGGGGAA'), (516, 'fwd', 'TAAG...",NaN,"[(623, 'fwd', 'GAAGGGAAA')]",HXT5,g1r1_m24_sc_para,KLLA0D13310g
2,Smik_8.173,Smik,"[(465, 'fwd', 'TAAGGGGAC'), (295, 'fwd', 'TTAG...",NaN,"[(668, 'fwd', 'AAAGGGAAA'), (535, 'fwd', 'CAAG...",HXT5,g1r1_m24_sc_para,KLLA0D13310g
3,YHR096C,Scer,"[(471, 'fwd', 'TTAGGGGAC'), (303, 'fwd', 'TTAG...","[(496, 'rev', 'CACGGGGAA')]","[(541, 'fwd', 'TAAGGGATG'), (184, 'rev', 'AGAG...",HXT5,g1r1_m24_sc_para,KLLA0D13310g
4,KAFR0B05060,Kafr,NaN,NaN,"[(69, 'rev', 'ACAGGGACG')]",NDE2,g1r1_m24_sc_para,KLLA0E21891g
5,CAGL0I00748g,Cgla,"[(356, 'rev', 'GCAGGGGAG')]","[(136, 'rev', 'TTCGGGGGG'), (224, 'rev', 'TGCG...","[(237, 'rev', 'GTAGGGAGC'), (489, 'rev', 'GCAG...",NDE2,g1r1_m24_sc_para,KLLA0E21891g
6,Suva_4.160,Suva,"[(137, 'rev', 'AAAGGGGAG'), (142, 'rev', 'GGAG...",NaN,"[(181, 'fwd', 'AAAGGGACT')]",NDE2,g1r1_m24_sc_para,KLLA0E21891g
7,Skud_13.306,Skud,NaN,"[(570, 'rev', 'GACGGGGTC'), (630, 'rev', 'GGCG...","[(520, 'fwd', 'ACAGGGATC'), (99, 'fwd', 'AGAGG...",NDE2,g1r1_m24_sc_para,KLLA0E21891g
8,Smik_4.151,Smik,"[(157, 'rev', 'GGAGGGGCT'), (172, 'rev', 'TTAG...",NaN,"[(153, 'rev', 'AAAGGGAGG'), (232, 'rev', 'TTAG...",NDE2,g1r1_m24_sc_para,KLLA0E21891g
9,YDL085W,Scer,"[(143, 'rev', 'GGAGGGGCT'), (158, 'rev', 'TTAG...","[(63, 'fwd', 'TGCGGGGAT')]","[(139, 'rev', 'AAAGGGAGG')]",NDE2,g1r1_m24_sc_para,KLLA0E21891g


In [37]:
new_output
full_output = pd.DataFrame()
pd.concat([full_output, new_output])

,spec,STRE,Msn2_pref,PDS,SC_common_name,category
KAFR0K01720,Kafr,"[(588, fwd, GAAGGGGTA)]",None,"[(1009, fwd, GCAGGGAGG), (353, rev, AGAGGGAAA)]",TOD6,other_para_low
CAGL0I05060g,Cgla,"[(2732, fwd, AAAGGGGAT), (547, fwd, GAAGGGGGT)...","[(1278, fwd, TTCGGGGCA), (1321, rev, TCCGGGGTT...","[(5047, fwd, ATAGGGATT), (4478, fwd, TGAGGGACA...",TOD6,other_para_low
Suva_5.202,Suva,None,None,None,TOD6,other_para_low
Skud_2.50,Skud,None,"[(595, fwd, GACGGGGTC)]","[(480, fwd, GCAGGGATT), (478, rev, CCAGGGAAA)]",TOD6,other_para_low
Smik_2.62,Smik,"[(576, fwd, TGAGGGGCT), (609, rev, TAAGGGGAT)]",None,None,TOD6,other_para_low
YBL054W,Scer,"[(600, fwd, GTAGGGGCT)]",None,"[(14, rev, CGAGGGAAC)]",TOD6,other_para_low


In [23]:
#print data to file
for gene in SC_common_name: 
    

,Unnamed: 0,SC_common_name,category
0,0,HXT5,g1r1_m24_sc_para
1,1,NDE2,g1r1_m24_sc_para
2,2,PIR3,g1r1_m24_sc_para
3,3,GND2,g1r1_m24_sc_para
4,4,RGI1,g1r1_m24_sc_para
5,5,USV1,g1r1_m24_sc_para
6,6,SOL4,g1r1_m24_sc_para
7,7,TSA2,g1r1_m24_sc_para
8,8,DBP1,g1r1_m24_sc_para
9,9,YJR115W,g1r1_m24_sc_para


In [2]:
motif_dict = {'STRE':'AGGGG',
             'Msn2_pref': 'CGGGG', 
              'PDS1': 'TTAGGGAT',
              'PDS2': 'TAAGGGAT',
              'PDS core': 'AGGGA'
             }
output_format = 'full'
sequence_context = 2
no_prom_thresh = 20 #if length of intergenic region is less than this, call no promoter

gene_name = 'SOL4'
gene = 'YGR248W'

#Lookup orthologs based on the YGOB Pillars dataframe
gene_pos = ''
for pos in ['p1','p2']: 
    Scer_pos = YGOB_pillars['Scer_' + pos]
    Scer_pos_find = Scer_pos[Scer_pos==gene]
    if len(Scer_pos_find)==1:
        if gene_pos != '':
            print('Warning: found gene in two pillars')
        gene_pos = pos
        gene_index = Scer_pos_find.index[0]
    elif len(Scer_pos_find)>1: 
        print('Warning: found gene twice in same pillar')

#extract intergenic regions from trevor's file

specs = ['Kafr','Cgla','Suva', 'Skud', 'Smik','Scer']
##to include pre wgd species would need to update
specs_pillar = [spec + '_' + gene_pos for spec in specs]

orthologs = YGOB_pillars.loc[gene_index, specs_pillar]


seqs = {}

for spec in specs: 
    fname_intergenics = os.path.normpath(data_processing_dir + 'promoter_phylogenies/intergenics/' + spec + '_intergenics.faa' )
    record_iterator = SeqIO.parse(fname_intergenics, "fasta")

    for seq in record_iterator: 
        if seq.id == orthologs[spec + '_' + gene_pos]:
            seqs[spec] = seq




#promoter_counts = io_library.exact_promoter_scan(gene_list, motif_dict, promoter_database, output_format, sequence_context)

#initial dataframe has species for index, but reindex on seq id before outputting
seq_ids = [seq.id for seq in seqs.values()]
output_data_frame = pd.DataFrame(index = seq_ids)
output_data_frame['spec'] = seqs.keys()

for motif_name, motif in motif_dict.items():
    L_motif = len(motif)
    output_motif = []

    for spec, seq in seqs.items(): 

        prom_seq = seq.seq
        if len(prom_seq)<no_prom_thresh: 
            output_motif.append('no_promoter')
        else: 
            output_motif_seq = io_library.exact_promoter_scan(motif, prom_seq, output_format, sequence_context)
            output_motif.append(output_motif_seq)
    output_data_frame[motif_name] = output_motif

output_data_frame

NameError: name 'YGOB_pillars' is not defined

In [7]:
seqs

{'Cgla': SeqRecord(seq=Seq('AGAAGATTGTAAAACCGGCAACTAGGAGAACAACATTTCAATAATCTCTATTTA...GTG', SingleLetterAlphabet()), id='CAGL0I02200g', name='CAGL0I02200g', description='CAGL0I02200g', dbxrefs=[]),
 'Kafr': SeqRecord(seq=Seq('AGTTTTCAAGCAACAATATTCGTATCTATATAGTCGGAAGTTTCAAAGTGATTT...CTA', SingleLetterAlphabet()), id='KAFR0B04400', name='KAFR0B04400', description='KAFR0B04400', dbxrefs=[]),
 'Scer': SeqRecord(seq=Seq('AGCATACATTTATGTGCACTGTAAGTAATTATATAGAATTTTTTTTTTTTTTTT...AAA', SingleLetterAlphabet()), id='YGR248W', name='YGR248W', description='YGR248W', dbxrefs=[]),
 'Skud': SeqRecord(seq=Seq('AGGGCACGTGTATGGATTGTGTAGGTAAATATATAGCAGGTTTTACAAGCTTGT...GAG', SingleLetterAlphabet()), id='Skud_7.581', name='Skud_7.581', description='Skud_7.581', dbxrefs=[]),
 'Smik': SeqRecord(seq=Seq('GTACACATGTATACATATTTTAAGTAATTTATGGCAAGGTTTTTCATATCTGTC...ACT', SingleLetterAlphabet()), id='Smik_16.59', name='Smik_16.59', description='Smik_16.59', dbxrefs=[]),
 'Suva': SeqRecord(seq=Seq('GCAGGACAGCAAGAGAA

In [15]:
gene_name = 'SOL3'
gene = 'YHR163W'

## Flag if no gene comes up
#Lookup orthologs based on the YGOB Pillars dataframe
gene_pos = ''
for pos in ['p1','p2']: 
    Scer_pos = YGOB_pillars['Scer_' + pos]
    Scer_pos_find = Scer_pos[Scer_pos==gene]
    if len(Scer_pos_find)==1:
        if gene_pos != '':
            print('Warning: found gene in two pillars')
        gene_pos = pos
        gene_index = Scer_pos_find.index[0]
    elif len(Scer_pos_find)>1: 
        print('Warning: found gene twice in same pillar')

#extract intergenic regions from trevor's file

specs = ['Kafr','Cgla','Suva', 'Skud', 'Smik','Scer']
##to include pre wgd species would need to update
specs_pillar = [spec + '_' + gene_pos for spec in specs]

orthologs = YGOB_pillars.loc[gene_index, specs_pillar]


seqs = {}

for spec in specs: 
    fname_intergenics = os.path.normpath(data_processing_dir + 'promoter_phylogenies/intergenics/' + spec + '_intergenics.faa' )
    record_iterator = SeqIO.parse(fname_intergenics, "fasta")

    for seq in record_iterator: 
        if seq.id == orthologs[spec + '_' + gene_pos]:
            seqs[spec] = seq


In [16]:
seq = seqs['Scer']

In [20]:
motif = 'AGGGG'
sequence_context = 2
output_format = 'full'
prom_seq = seq.seq

#exact_promoter_scan(motif, prom_seq, output_format='count', sequence_context=0):
#this function is for a single sequence - same name used to be for one that searched a list of genes 
# now called exact_promoter_scan_genelist
# 
# scans a sequence (biopython object) for a motif in both forward and backward directions 
# If output format is 'count', then just lists the number of hits. 
# If output format is 'full', gives the location, direction (relative to the gene), and
# sequence context (number of bases on either side of the hit)output_format


L_motif = len(motif)

#fwd search
prom_seq_fwd = str(prom_seq)
L_prom = len(prom_seq_fwd)
motif_sites_fwd = [m.start() for m in re.finditer(motif, prom_seq_fwd)]
#to find overlapping motifs
#[m.start() fr m in re.finditer('(?=' + motif + ')', prom_seq_rev)]

#reverse search
#rev search
prom_seq_rev = str(prom_seq.reverse_complement())
#location is in reference to the fwd seq
motif_sites_rev = [len(prom_seq_rev) - m.start() - L_motif for m in re.finditer(motif, prom_seq_rev)]

if output_format == 'count':  
    n_motifs_gene = len(motif_sites_fwd) + len(motif_sites_rev)
    output = n_motifs_gene
elif output_format == 'full': 
    loc_hitseq_gene = []
    for motif_site in motif_sites_fwd: 
        loc = L_prom-1-motif_site
        hitseq = prom_seq_fwd[motif_site-sequence_context:motif_site+L_motif+sequence_context]  
        hitdir = 'fwd'
        loc_hitseq_gene.append((loc,hitdir,hitseq))
    for motif_site in motif_sites_rev: 
        loc = L_prom - motif_site+L_motif
        hitseq = prom_seq_rev[(L_prom-motif_site-L_motif-sequence_context):(L_prom-motif_site+sequence_context)]
        hitdir = 'rev'
        loc_hitseq_gene.append((loc,hitdir,hitseq))

    if len(loc_hitseq_gene)==0: 
        output = None
    else:
        output = loc_hitseq_gene
else: 
    print("choose output_format: 'count' or 'full' ")
        


In [21]:
output


[(23, 'rev', 'CCAGGGGTA')]

In [23]:
prom_seq_fwd

'AACGCCGAATTTACAACTCGAAACTTTGCTTTGTCAAAGATATGTACGTAAGCATATATATCCGGATTGTATAATATTCATAAAATAGTAAAGTATATTTTCTTTGCGCGGTGTTATAATTTGCTCTTTTGCATTTTTCCGTTTTTTTCTTACCTTTTTATCCCCGACAATGGTAATAGTGGTATATTATATAAAGAGGAAAGGCCTCGAGGATAATAGAAGGCAATGCACCATCAATTGCTTTACCCCTGGTCCGCGACCAA'

In [24]:
prom_seq_rev

'TTGGTCGCGGACCAGGGGTAAAGCAATTGATGGTGCATTGCCTTCTATTATCCTCGAGGCCTTTCCTCTTTATATAATATACCACTATTACCATTGTCGGGGATAAAAAGGTAAGAAAAAAACGGAAAAATGCAAAAGAGCAAATTATAACACCGCGCAAAGAAAATATACTTTACTATTTTATGAATATTATACAATCCGGATATATATGCTTACGTACATATCTTTGACAAAGCAAAGTTTCGAGTTGTAAATTCGGCGTT'

In [26]:
motif

'AGGGG'

In [25]:
[len(prom_seq_rev) - m.start() - L_motif for m in re.finditer(motif, prom_seq_rev)]

[245]

In [122]:
#suspect Smik gnd2 is actually Smik gnd1 - check intergenics
#RGI1 overlaps with dubious orf.  
#Smik 4.730 is missing from intergenics.  I think that intergenics is not as good as 700bp upstream  
#YER067W overlaps with dubious orf so isn't in trevor's file.  If you take the promoter as the 1kb upstream then it there are several STREs and one PDS. 


gene_names

['HXT5',
 'NDE2',
 'PIR3',
 'GND2',
 'RGI1',
 'USV1',
 'SOL4',
 'TSA2',
 'DBP1',
 'YJR115W',
 'YHR097C',
 'MBR1',
 'GND1',
 'HSP150',
 'NDE1',
 'HXT3',
 'RGI2',
 'DED1',
 'PAL1',
 'ECM13',
 'SOL3',
 'RGM1',
 'TSA1',
 'ISF1',
 'GSY2',
 'YBR241C',
 'USV1',
 'GLG1',
 'ISF1',
 'MRK1',
 'ATG33',
 'FMP45',
 'TSL1',
 'PGM2',
 'GSY1',
 'YNL194C',
 'RGM1',
 'VPS73',
 'RIM11',
 'TPS3',
 'GLG2',
 'SCM4',
 'PGM1',
 'MBR1',
 'HSP12',
 'GOR1',
 'YMR196W',
 'PEX18',
 'SHH4',
 'SSE2',
 'YJL045W',
 'SHH3',
 'PEX21',
 'SDH3',
 'SDH1',
 'SDH4',
 'SSE1',
 'YBR139W',
 'SDS24',
 'SDS23',
 'CYC7',
 'HXK1',
 'UBI4',
 'CTT1',
 'GSP2',
 'HXK2',
 'GSP1',
 'CYC1',
 'GIS1',
 'DOT6',
 'RPH1',
 'TOD6']

In [144]:
output_dict['GSP2']

,spec,STRE,Msn2_pref,PDS
Suva_10.389,Suva,"[(229, rev, TTAGGGGAG)]",None,"[(234, rev, GGAGGGACA)]"
Skud_15.352,Skud,"[(487, fwd, CAAGGGGCA), (458, fwd, CAAGGGGAG),...",None,"[(439, fwd, TAAGGGAGG), (711, rev, AAAGGGAAT)]"
Smik_12.371,Smik,None,None,"[(65, rev, TGAGGGAAA), (271, rev, AAAGGGATC)]"
YOR185C,Scer,"[(482, fwd, TAAGGGGCC), (150, rev, TAAGGGGTG),...",None,"[(525, fwd, CCAGGGAAT), (473, fwd, ACAGGGAAA),..."


In [88]:
output_dict[io_library.SC_orf_lookup_by_name(['NDE2'])[0]]

KeyError: 'YDL085W'

In [58]:
orthologs = YGOB_pillars.loc[gene_index, specs]



In [56]:
a = next(record_iterator)

In [63]:
#find and extract all instances of STREs and PDSs


seqs

{'Cgla': SeqRecord(seq=Seq('TATGTGTTTCCACCAACTAAAAGAAAAAACTTGCCAAGAAGATACTTTCAACTA...GCA', SingleLetterAlphabet()), id='CAGL0L11880g', name='CAGL0L11880g', description='CAGL0L11880g', dbxrefs=[]),
 'Kafr': SeqRecord(seq=Seq('GTATGGGGGAGGGAAGTGGGTTTTGGAGTGTTATTGCGTCGAGTAAATATATAT...AAG', SingleLetterAlphabet()), id='KAFR0A02380', name='KAFR0A02380', description='KAFR0A02380', dbxrefs=[]),
 'Scer': SeqRecord(seq=Seq('TTCTCTTGTCTTACAGGTCCCTTGAATTGGCATTTACGTTTACATCCGATCCGG...ACA', SingleLetterAlphabet()), id='YDR096W', name='YDR096W', description='YDR096W', dbxrefs=[]),
 'Skud': SeqRecord(seq=Seq('ACGATAACATTTTGTTCTTTTGTACCATGTATAAGCAATGTCATATGATATAGA...ATT', SingleLetterAlphabet()), id='Skud_4.354', name='Skud_4.354', description='Skud_4.354', dbxrefs=[]),
 'Smik': SeqRecord(seq=Seq('ATGATAACATTTTTTGTTCTTTTGTACTATGTATAACAAATGTCATATGATATA...ATA', SingleLetterAlphabet()), id='Smik_4.337', name='Smik_4.337', description='Smik_4.337', dbxrefs=[]),
 'Suva': SeqRecord(seq=Seq('TACACACACGAGCAACA

In [64]:
seqs

{'Cgla': SeqRecord(seq=Seq('TATGTGTTTCCACCAACTAAAAGAAAAAACTTGCCAAGAAGATACTTTCAACTA...GCA', SingleLetterAlphabet()), id='CAGL0L11880g', name='CAGL0L11880g', description='CAGL0L11880g', dbxrefs=[]),
 'Kafr': SeqRecord(seq=Seq('GTATGGGGGAGGGAAGTGGGTTTTGGAGTGTTATTGCGTCGAGTAAATATATAT...AAG', SingleLetterAlphabet()), id='KAFR0A02380', name='KAFR0A02380', description='KAFR0A02380', dbxrefs=[]),
 'Scer': SeqRecord(seq=Seq('TTCTCTTGTCTTACAGGTCCCTTGAATTGGCATTTACGTTTACATCCGATCCGG...ACA', SingleLetterAlphabet()), id='YDR096W', name='YDR096W', description='YDR096W', dbxrefs=[]),
 'Skud': SeqRecord(seq=Seq('ACGATAACATTTTGTTCTTTTGTACCATGTATAAGCAATGTCATATGATATAGA...ATT', SingleLetterAlphabet()), id='Skud_4.354', name='Skud_4.354', description='Skud_4.354', dbxrefs=[]),
 'Smik': SeqRecord(seq=Seq('ATGATAACATTTTTTGTTCTTTTGTACTATGTATAACAAATGTCATATGATATA...ATA', SingleLetterAlphabet()), id='Smik_4.337', name='Smik_4.337', description='Smik_4.337', dbxrefs=[]),
 'Suva': SeqRecord(seq=Seq('TACACACACGAGCAACA

,spec,STRE,Msn2_pref,PDS
KAFR0B04400,Kafr,None,None,None
CAGL0I02200g,Cgla,"[(151, rev, GCAGGGGAA), (190, rev, AAAGGGGTG)]",None,"[(353, rev, CAAGGGATT)]"
Suva_7.543,Suva,"[(148, rev, TTAGGGGGA)]","[(21, rev, CTCGGGGTG)]","[(182, fwd, TAAGGGATT), (175, rev, CAAGGGATA)]"
Skud_7.581,Skud,"[(47, rev, TTAGGGGGG)]",None,"[(70, fwd, CAAGGGATA), (63, rev, CAAGGGATT)]"
Smik_16.59,Smik,"[(135, rev, TTAGGGGGT)]",None,"[(160, fwd, CAAGGGATA), (154, rev, CAAGGGATT)]"
YGR248W,Scer,"[(131, rev, TTAGGGGGG)]",None,"[(160, fwd, CAAGGGATG), (155, rev, CAAGGGATT)]"


In [ ]:
def write_YGOB_orth_lookup_table(species1, species2, base_dir, all_ortholog_file):
    #for each position in species 1
    #Assign orthologs or 'NONE' to each column from Position 2 and 3
    fname = os.path.normpath(base_dir + all_ortholog_file)
    orth_positions = {'Kluyveromyces lactis': [15], 'Saccharomyces cerevisiae' : [11,21]}
    #YGOB_Pillars.txt order of species: 
    #    0    V. polyspora Position 1
    #    1    T. phaffii Position 1
    #    2    T. blattae Position 1
    #    3    N. dairenensis Position 1
    #    4    N. castellii Position 1
    #    5    K. naganishii Position 1
    #    6    K. africana Position 1
    #    7    C. glabrata Position 1
    #    8    S. bayanus var. uvarum Position 1
    #    9    S. kudriavzevii Position 1
    #    10   S. mikatae Position 1
    #    11   S. cerevisiae Position 1
    #    12   Ancestral Gene Order
    #    13   Z. rouxii
    #    14   T. delbrueckii
    #    15   K. lactis
    #    16   E. gossypii 
    #    17   E. cymbalariae
    #    18   L. kluyveri
    #    19   L. thermotolerans
    #    20   L. waltii
    #    21   S. cerevisiae Position 2
    #    22   S. mikatae Position 2
    #    23   S. kudriavzevii Position 2
    #    24   S. bayanus var. uvarum Position 2
    #    25   C. glabrata Position 2
    #    26   K. africana Position 2
    #    27   K. naganishii Position 2
    #    28   N. castellii Position 2
    #    29   N. dairenensis Position 2
    #    30   T. blattae Position 2
    #    31   T. phaffii Position 2
    #    32   V. polyspora Position 2
    
    species1_columns = orth_positions[species1]
    species2_columns = orth_positions[species2]
    
    with open(fname) as f:
        orth_lookup = []
        for line in f:
            linesp = line.split()
            for column1 in species1_columns: 
                if linesp[column1]!= '---':
                    species1_gene = [linesp[column1]]
                    species2_genes = []
                    for column2 in species2_columns: 
                        if linesp[column2]!='---': 
                            species2_genes.append(linesp[column2])
                    if len(species2_genes) == 0:
                        species2_genes.append('NONE')
                    orth_lookup.append(species1_gene + species2_genes)
                    
    orth_file_abbrev = {'Kluyveromyces lactis': 'Klac', 'Saccharomyces cerevisiae': 'Scer', 'Candida glabrata':'Cgla', 'Saccharomyces castellii' : 'Scas', 'Saccharomyces bayanus' : 'Sbay'}
    orth_lookup_outputfname = os.path.normpath(base_dir + '\expression_data\ortholog_files_YGOB\\' + orth_file_abbrev[species1] + "-" + orth_file_abbrev[species2] + "-orthologs.txt"  )
    orth_lookup_outputfile = open(orth_lookup_outputfname, 'w')
    
    for gene in orth_lookup:
        line = '\t'.join(gene)+'\n'
        orth_lookup_outputfile.write(line)
    orth_lookup_outputfile.close()
    
    return orth_lookup 


## Motif phylogenies for Nov 2018 CTTE meeting


LIst of genes to look at: 

	M24
		Paralogs: GND2, SOL4, TKL2 (PPP) , TSA1, SHH3, PIR3, YCL048W-A, YDR034W-B, SHH4, RTN2
		Others: CYB2, PUT4
	Rph1
		Paralogs: CYC7, YSC84, GSP2, SSE2, GPD1, SHC1, TPK1
		Others: ZWF1 (1st step in PPP), YTP1, TPK2
	Both
		Paralogs: TSL1, RGI1
		Others: CTT1, EGO4, RTC3, COX26


GPA1/PRM3 (mating related) either gained control by Msn2/4 in KL or lost it in SC 
Similarly Fus3 (mating related) re Rph1
	Phenotype: KL require Msn2/4 for mating phenotype (is this already known?)




SIP1 and GRE1  
 under Rph1/Gis1 in KL - in fact Rph1/Gis1 deletion causes even bigger response in KL under PKA inhibition (so PKA inhibition doesn't fully inhibit Rph1 in KL)  msn2/4 deletion does not change response to drug. 
 under Msn2/4 control in S.Cer.  Rph1/Gis1 deletion may cause a very small incresase in SIP18.  Fairly big increase in GRE1 under Rph1/Gis1

Similar: GPH1, SPG4, SPI1 (which has paralog), also check YNL194C



In [ ]:
List of motifs to follow: 
    

    motif_dict = {'STRE':'AGGGG',
             'Msn2_pref': 'CGGGG', 
              'PDS1': 'TTAGGGAT',
              'PDS2': 'TAAGGGAT',
              'PDS core': 'AGGGA'
             }
    
    TATA: 
        KL
        
MOTIF TATAHATA DREME-4
#             Word    RC Word        Pos        Neg    P-value    E-value
# BEST    TATAHATA   TATDTATA        175       1522   4.5e-009   1.9e-004
#         TATAAATA   TATTTATA         72        605   2.9e-004   1.2e+001
#         TATATATA   TATATATA         89        797   3.7e-004   1.6e+001
#         TATACATA   TATGTATA         51        415   1.3e-003   5.5e+001
letter-probability matrix: alength= 4 w= 8 nsites= 250 E= 1.9e-004
0.000000 0.000000 0.000000 1.000000
1.000000 0.000000 0.000000 0.000000
0.000000 0.000000 0.000000 1.000000
1.000000 0.000000 0.000000 0.000000
0.312000 0.220000 0.000000 0.468000
1.000000 0.000000 0.000000 0.000000
0.000000 0.000000 0.000000 1.000000
1.000000 0.000000 0.000000 0.000000

SC
#             Word    RC Word        Pos        Neg    P-value    E-value
# BEST    CWTATATA   TATATAWG        159       1368   3.0e-008   1.3e-003
#         CTTATATA   TATATAAG         71        550   2.4e-005   1.0e+000
#         CATATATA   TATATATG         96        946   3.0e-003   1.3e+002
letter-probability matrix: alength= 4 w= 8 nsites= 188 E= 1.3e-003
0.000000 1.000000 0.000000 0.000000
0.638298 0.000000 0.000000 0.361702
0.000000 0.000000 0.000000 1.000000
1.000000 0.000000 0.000000 0.000000
0.000000 0.000000 0.000000 1.000000
1.000000 0.000000 0.000000 0.000000
0.000000 0.000000 0.000000 1.000000
1.000000 0.000000 0.000000 0.000000


KL mystery motif
MOTIF TTTGCTW DREME-2
#             Word    RC Word        Pos        Neg    P-value    E-value
# BEST     TTTGCTW    WAGCAAA        176       1403   3.2e-012   1.3e-007
#          TTTGCTT    AAGCAAA        123        954   1.0e-008   4.4e-004
#          TTTGCTA    TAGCAAA         78        563   1.0e-006   4.3e-002
letter-probability matrix: alength= 4 w= 7 nsites= 235 E= 1.3e-007
0.000000 0.000000 0.000000 1.000000
0.000000 0.000000 0.000000 1.000000
0.000000 0.000000 0.000000 1.000000
0.000000 0.000000 1.000000 0.000000
0.000000 1.000000 0.000000 0.000000
0.000000 0.000000 0.000000 1.000000
0.382979 0.000000 0.000000 0.617021

Jaspar NHP6B / 6A
